In [1]:
%load_ext autoreload
%autoreload 2

import stravaio
import sweat
import dotenv
import pandas as pd
import plotly_express as px
dotenv.load_dotenv()
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from sweat.metrics import rolling_mean

In [3]:
token = stravaio.strava_oauth2()

2019-07-10 15:17:58.055 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2019-07-10 15:18:03.204 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: 586cac68db346c21195b313a10e9fbcb7ca0bc18
2019-07-10 15:18:04.841 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: d5c20300fc6f405d1c92eaaab468b1aa3a9b32f7


In [4]:
access_token= token['access_token']

In [5]:
client = stravaio.StravaIO(access_token)

In [6]:
athlete = client.get_logged_in_athlete()

In [7]:
list_activities = client.get_logged_in_athlete_activities(after="2019-06-25")

Fetched 12, the latests is on 2019-07-10 04:11:47+00:00


In [8]:
[print(f"{a.id}: {a.name}") for a in list_activities]

2478368148: Morning Ride
2484463522: One Hour Lower Back Routine
2488860384: Morning Ride
2491572855: Morning Ride
2494619456: One Hour Lower Back Routine
2497323866: SUF Strength - Intermediate Level 3A
2499770891: Morning Ride
2506439358: SUF Strength - Intermediate Level 3B
2507832213: Morning Ride
2510630926: Morning Ride
2517428229: One Hour Lower Back Routine
2519436384: Morning Yoga Routine


[None, None, None, None, None, None, None, None, None, None, None, None]

In [9]:
id = 2510630926
activity = client.get_activity_by_id(id)
stream = client.get_activity_streams(athlete_id=athlete.id, id=id)
df = pd.DataFrame(stream.to_dict())
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng
0,299,1396.8,380.0,5.4,110,82,151,18,True,1.9,47.086426,15.424882
1,300,1402.2,380.2,4.3,110,82,113,18,True,1.9,47.086469,15.424850
2,301,1407.2,380.2,4.8,110,83,84,18,True,2.0,47.086506,15.424812
3,302,1412.0,380.2,5.2,110,83,0,18,True,1.0,47.086536,15.424767
4,303,1416.8,380.4,5.1,111,83,0,18,True,0.8,47.086564,15.424718


# Preprocess

In [10]:
window=30
df['smooth_watts'] = sweat.metrics.rolling_mean(df.watts, window)
df['smooth_heartrate'] = sweat.metrics.rolling_mean(df.heartrate, window)
df['smooth_cadence'] = sweat.metrics.rolling_mean(df.cadence, window)

In [11]:
e4DP = [-1, 200, 257, 326, 444, 983]
e4DP_names = ['END', 'FTP', 'MAP', 'AC', 'NM']
e4DP_colors = [None,
               px.colors.sequential.dense[1],
               px.colors.sequential.dense[3],
               px.colors.sequential.dense[6],
               px.colors.sequential.dense[8],
               px.colors.sequential.dense[11]]

In [12]:
df['power_zones'] = sweat.metrics.compute_zones(df.smooth_watts, zones=e4DP)

In [13]:
time_in_power_zones = sweat.metrics.time_in_zones(df.smooth_watts, zones=e4DP)
time_in_power_zones

array([8206, 1878,  286,    4,    0], dtype=int64)

In [14]:
df['hr_zones'] = sweat.metrics.compute_zones(df.smooth_heartrate, lthr=165)

In [15]:
time_in_hr_zones = sweat.metrics.time_in_zones(df.smooth_heartrate, lthr=165)
time_in_hr_zones

array([3850, 4094, 1975,  455,    0], dtype=int64)

# Plot

In [34]:
from velo_plotly import ride_analysis

In [35]:
fig = ride_analysis(df)

In [36]:
iplot(fig)